<a href="https://colab.research.google.com/github/MariaPompiani/basic-basket-2025-analysis/blob/main/extensao2_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import pandas as pd
import matplotlib.pyplot as plt

In [28]:
data = pd.read_excel('/content/drive/MyDrive/ICB_2s-2025.xlsx')
data

,Data_Coleta,Estabelecimento,Produto,Marca,Preco,Quantidade,PPK
0,2022-09-12,SUP-0,Leite,Parmalat,6.28,1.0,6.280000
1,2022-09-12,SUP-1,Tomate,NaN,6.99,1.0,6.990000
2,2022-09-12,SUP-2,Arroz,Namorado,17.98,5.0,3.596000
3,2022-09-12,SUP-2,Arroz,Broto Legal,22.69,5.0,4.538000
4,2022-09-12,SUP-0,Feijão,Broto Legal,10.98,1.0,10.980000
...,...,...,...,...,...,...,...
34027,2025-09-22,SUP-2,Manteiga,Outro,17.99,0.2,89.950000
34028,2025-09-22,SUP-0,Café,Caboclo,29.95,0.5,59.900000
34029,2025-09-22,SUP-2,Leite,Ninho,7.99,1.0,7.990000
34030,2025-09-22,SUP-0,Arroz,Empório São João,29.99,5.0,5.998000


In [30]:
data.describe()

,Data_Coleta,Preco,Quantidade,PPK
count,34032,34032.000000,34032.000000,34032.000000
mean,2024-04-02 13:55:46.121297664,13.780066,1.379653,19.659848
min,2022-09-12 00:00:00,1.190000,0.200000,0.449500
25%,2023-06-20 00:00:00,5.900000,0.900000,5.583000
50%,2024-04-10 00:00:00,8.990000,1.000000,7.655556
75%,2025-01-20 00:00:00,18.380000,1.000000,29.960000
max,2025-09-22 00:00:00,64.990000,60.000000,147.500000
std,NaN,10.581267,1.662825,22.913911


In [31]:
data.describe(include='object')

,Estabelecimento,Produto,Marca
count,34032,34032,27637
unique,41,27,168
top,SUP-21,Café,Broto Legal
freq,1362,4122,1570


In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34032 entries, 0 to 34031
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Data_Coleta      34032 non-null  datetime64[ns]
 1   Estabelecimento  34032 non-null  object        
 2   Produto          34032 non-null  object        
 3   Marca            27637 non-null  object        
 4   Preco            34032 non-null  float64       
 5   Quantidade       34032 non-null  float64       
 6   PPK              34032 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 1.8+ MB


# **1. Somente a variavel marca tem produtos nulos, como vamos tratar isso?**
- todos que forem nulos recebem um valor novo para marca, tipo 'sem marca'

# **Perguntas**

**1. Previsão de preços futuros de todos os produtos no geral**
- separar cada produto, ver o comportamento deles.
- realizar os tratamentos para aplicar os algoritmos de series temporais
- analisar o erro
- fazer a previsão



In [33]:
data['Produto'].nunique()

27

Notamos que alguns valores aparecem de forma provavelmente duplicados. Como farinha e farinha de trigo. Para isso iremos investigar.

In [34]:
data['Produto'].value_counts()

,count
Produto,
Café,4122
Arroz,4015
Manteiga,3645
Feijão,3602
Leite,3490
Farinha,3199
Óleo,2466
Açúcar,2420
Tomate,1027


- Farinha será considerado Farinha de Trigo
- Carne Coxão Mole será Carne Bovina Coxão Mole
- Carne Pernil será Carne Suina Pernil
- Parne Acem e Carne Acém será Carne Bovina Acem
- Pão sera Pao Frances

In [35]:
data['Produto'] = data['Produto'].replace('Farinha', 'Farinha de Trigo')
data['Produto'] = data['Produto'].replace('Carne Coxão Mole', 'Carne Bovina Coxão Mole')
data['Produto'] = data['Produto'].replace('Carne Pernil', 'Carne Suína Pernil')
data['Produto'] = data['Produto'].replace('Carne Acém', 'Carne Bovina Acem')
data['Produto'] = data['Produto'].replace('Carne Acem', 'Carne Bovina Acem')
data['Produto'] = data['Produto'].replace('Pão', 'Pão Francês')
data['Produto'] = data['Produto'].replace('Macarrão com Ovos', 'Macarrão')

In [36]:
data['Produto'].value_counts()

,count
Produto,
Café,4122
Arroz,4015
Manteiga,3645
Feijão,3602
Leite,3490
Farinha de Trigo,3271
Óleo,2466
Açúcar,2420
Tomate,1027


Os nomes dos produtos foram atualizados para pertercerem ao mesmo grupo e manterem o padrão.

In [37]:
data.head()

,Data_Coleta,Estabelecimento,Produto,Marca,Preco,Quantidade,PPK
0,2022-09-12,SUP-0,Leite,Parmalat,6.28,1.0,6.280
1,2022-09-12,SUP-1,Tomate,NaN,6.99,1.0,6.990
2,2022-09-12,SUP-2,Arroz,Namorado,17.98,5.0,3.596
3,2022-09-12,SUP-2,Arroz,Broto Legal,22.69,5.0,4.538
4,2022-09-12,SUP-0,Feijão,Broto Legal,10.98,1.0,10.980


In [ ]:
# Média de cada produto em todas as marcas do preço por kilo durante todos os dias. Tambem o desvio padrão para ver a variação